# **Analysing LAEI data**

### As we are using a 20-metre spatial granularity dataset, you are going to encounter a lot of data points for each of the pollutants concerned. Now, python provides multiple libraries for visualising geospatial data. For example, folium, ipyleaflet, rasterio, geoplot, bokeh and many more. Currently, we are more interested in publishing interactive visualisations. One python library that provides this support is "Bokeh" that targets web browsers for representation. Let's see what we can do with Bokeh.

In [ ]:
import geopandas as gpd
import pandas as pd
import numpy as np

In [ ]:
import os
os.listdir()

In [ ]:
df = pd.read_csv("laei_LAEI2019_2025_Base_Final_CorNOx15_NO2.csv")
gdf = gpd.GeoDataFrame(df,geometry=gpd.points_from_xy(df["x"], df["y"], crs="epsg:4326"))

In [ ]:
from convertbng.util import convert_lonlat
# res = convert_lonlat((gdf.x.min(),gdf.x.max()), (gdf.y.min(),gdf.y.max()))
# res
gdf["Lon"],gdf["Lat"] = convert_lonlat(gdf["x"].tolist(), gdf["y"].tolist())

In [ ]:
gdf =gdf.drop('geometry',axis=1)
gdf = gpd.GeoDataFrame(gdf, geometry=gpd.points_from_xy(gdf.Lat, gdf.Lon))

In [ ]:
# """Converts decimal longitude/latitude to Web Mercator format"""
def wgs84_to_web_mercator(df, lon="Lon", lat="Lat"):
    k = 6378137
    df["x_wm"] = df[lon] * (k * np.pi/180.0)
    df["y_wm"] = np.log(np.tan((90 + df[lat]) * np.pi/360.0)) * k
    return df

In [ ]:
gdf = wgs84_to_web_mercator(gdf)

In [ ]:
gdf.to_csv('NO2_transformed.csv')

## **Loading Data**

### Please download the [**emission data**](https://data.london.gov.uk/download/london-atmospheric-emissions-inventory--laei--2019/f9f70e47-5683-4430-86e2-ffa2a50b31b1/LAEI2019-Concentrations-Data-CSV.zip "**emission data**"). Now load your data as a _geopandas_ dataframe. Make sure you are using appropriate referencing system.

In [ ]:
df = pd.read_csv('NO2_transformed.csv')
gdf = gpd.GeoDataFrame(df,geometry=gpd.points_from_xy(df["Lon"], df["Lat"], crs="epsg:4326"))

## **Visualising the Data**

### We will use _Bokeh_ plots to visualise the data. Make sure you have _Bokeh_ installed in your PC/laptop. Since _Bokeh_ generates interactive visualisation, you always need to run the _output_notebook_ function to load the library in jupyter.

In [ ]:
from bokeh.models import BoxZoomTool
from bokeh.plotting import figure, output_notebook, show

output_notebook()

### Bokeh is compatible with several XYZ tile services that use the Web Mercator projection. If you are using the inbuilt tile services, make sure to transform your projection to Web Mercator format. Once, you have the appropriate format, you need to specify the plot extent. Here, we are going to use the dataframe to specify the plot extent.

In [ ]:
# import numpy as np

# def wgs84_to_web_mercator(coor):
# # """Converts decimal longitude/latitude to Web Mercator format"""
#     lon=coor[0]
#     lat=coor[1]
#     k = 6378137
#     x = lon * (k * np.pi/180.0)
#     y = np.log(np.tan((90 + lat) * np.pi/360.0)) * k
#     return x,y

In [ ]:
# Define parameters to create the plot area
# London = X,Y = ((-60888.862704380386,38169.921636820196),(6667658.647822232,6747198.269554087))
London = X,Y = ((gdf.x_wm.min(),gdf.x_wm.max()),(gdf.y_wm.min(),gdf.y_wm.max()))
plot_width = int(990)
plot_height = int(plot_width//1.2)

options = dict(line_color=None, fill_color='blue',size=5)

### You can now create your own functions to generate plots using _Bokeh_.

In [ ]:
# Functions to create simple plots
def simple_plot(tools='pan,wheel_zoom,reset',plot_width=plot_width,plot_height=plot_height,**plot_kwargs):
    p = figure(title = 'NO2 Concentration from LAEI',tools=tools, width=plot_width,height=plot_height,
               x_range=X, y_range=Y,outline_line_color=None,
               min_border=0,min_border_left=0,min_border_right=0,min_border_top=0,
               min_border_bottom=0,**plot_kwargs)
    p.axis.visible = False
    p.xgrid.grid_line_color = None
    p.ygrid.grid_line_color = None

    p.add_tools(BoxZoomTool(match_aspect=True))

    return p

In [ ]:
from bokeh.tile_providers import Vendors
import xyzservices.providers as xyz

In [ ]:
%%time
samples = gdf.sample(n=100000)
p= simple_plot()
p.add_tile(Vendors.STAMEN_TERRAIN_RETINA)
p.circle(x=samples["x_wm"],y=samples["y_wm"], **options)

show(p)

In [ ]:
gdf = gdf.drop('geometry',axis=1)

### Now this plot doesn't reveal much about the concentration. We are more interested in looking at the concentration values. Colormaps can represent the concentration better.

In [ ]:
%%time
from bokeh.palettes import PRGn, RdYlGn
from bokeh.plotting import ColumnDataSource
from bokeh.transform import linear_cmap
from bokeh.models import LinearColorMapper, ColorBar, NumeralTickFormatter

# Choose palette
palette = RdYlGn[11]

samples = gdf.sample(n=10000)

source = ColumnDataSource(data=samples)
color_mapper = linear_cmap(field_name = 'conc', palette = palette, low = samples['conc'].min(), high = samples['conc'].max())
options = dict(source=source,line_color=None, color=color_mapper,size=5)
p= simple_plot()
p.add_tile(Vendors.CARTODBPOSITRON)
p.circle(x='x_wm',y='y_wm', **options)

#Defines color bar
color_bar = ColorBar(color_mapper=color_mapper['transform'], 
                     formatter = NumeralTickFormatter(format='0.0[0000]'), 
                     label_standoff = 13, width=8, location=(0,0))
# Set color_bar location
p.add_layout(color_bar, 'right')

show(p)

## _Undersampling_

In [ ]:
samples = gdf.sample(n=10000)

source = ColumnDataSource(data=samples)
color_mapper = linear_cmap(field_name = 'conc', palette = palette, low = samples['conc'].min(), high = samples['conc'].max())
options = dict(source=source,line_color=None, color=color_mapper,size=5)
p= simple_plot()
p.add_tile(Vendors.CARTODBPOSITRON)
p.circle(x='x_wm',y='y_wm', **options)

#Defines color bar
color_bar = ColorBar(color_mapper=color_mapper['transform'], 
                     formatter = NumeralTickFormatter(format='0.0[0000]'), 
                     label_standoff = 13, width=8, location=(0,0))
# Set color_bar location
p.add_layout(color_bar, 'right')

show(p)

## _Overplotting_

In [ ]:
%%time
samples = gdf.sample(n=1000000)

source = ColumnDataSource(data=samples)
color_mapper = linear_cmap(field_name = 'conc', palette = palette, low = samples['conc'].min(), high = samples['conc'].max())
options = dict(source=source,line_color=None, color=color_mapper,size=5)
p= simple_plot()
p.add_tile(Vendors.CARTODBPOSITRON)
p.circle(x='x_wm',y='y_wm', **options)

#Defines color bar
color_bar = ColorBar(color_mapper=color_mapper['transform'], 
                     formatter = NumeralTickFormatter(format='0.0[0000]'), 
                     label_standoff = 13, width=8, location=(0,0))
# Set color_bar location
p.add_layout(color_bar, 'right')

show(p)

### Now we hardly can differentiate between the concentration levels across the plot area. Such overplotting occurs mostly for large datasets. We can use _datashader_ package for meaningful visualisations. Datashader breaks the creation of images of data into 3 main steps:

### 1. Projection - Each record is projected into zero or more bins of a nominal plotting grid shape, based on a specified glyph. We will first create create a Canvas object with the shape of the eventual plot.

### 2. Aggregation - Reductions are computed for each bin, compressing the potentially large dataset into a much smaller aggregate array.

### 3. Transformation - These aggregates are then further processed, eventually creating an image where we will map the resulting counts into a visible color from a specified range.

### Click this [link](https://datashader.org/getting_started/index.html) to read more about _datashader_.

In [ ]:
import datashader as ds
from datashader import transfer_functions as tf
from functools import partial
from datashader.utils import export_image
from datashader.colors import colormap_select, Hot, inferno

In [ ]:
import colorcet as cc

In [ ]:
gdf.head()

In [ ]:
# cool trick (from the offical notebook to make function calls more elegant)
background = "black"
export = partial(export_image, export_path="export", background="black")
cm = partial(colormap_select, reverse=(background!="black"))

# this functions wraps the image creating process of datashader
def create_image(x_range, y_range, w=plot_width, h=plot_height):
    cvs = ds.Canvas(plot_width=w, plot_height=h, x_range=x_range, y_range=y_range)
    agg = cvs.points(gdf, 'x_wm', 'y_wm',ds.mean('conc'))
    img = tf.shade(agg, cmap=cm(Hot),how='eq_hist')
    return tf.dynspread(img, threshold=0.5, max_px=4)

In [ ]:
X,Y = ((gdf.x_wm.min(),gdf.x_wm.max()),(gdf.y_wm.min(),gdf.y_wm.max()))

In [ ]:
export(create_image(X,Y,plot_width,plot_height),"NO2 concentration")

In [ ]:
gdf = gdf[gdf['conc']<40]

In [ ]:
from distfit import distfit
dfit = distfit()

In [ ]:
p = figure(width=670, height=400, toolbar_location=None,
           title="Distribution of emission concentration levels")

# Histogram
bins = np.linspace(gdf.conc.min(), gdf.conc.max(), 40)
hist, edges = np.histogram(gdf['conc'], density=True, bins=bins)
p.quad(top=hist, bottom=0, left=edges[:-1], right=edges[1:],
         fill_color="skyblue", line_color="white",
         legend_label="Distribution of NO2 Emission concentration levels")

# # Probability density function
# x = np.linspace(gdf.conc.min(), gdf.conc.max(), 100)
# pdf = np.exp(-0.5*x**2) / np.sqrt(2.0*np.pi)
# p.line(x, pdf, line_width=2, line_color="navy",
#        legend_label="Probability Density Function")

p.y_range.start = 0
p.xaxis.axis_label = "NO2_Concentration"
p.yaxis.axis_label = "PDF(NO2_Concentration)"

show(p)

In [ ]:
len(gdf[gdf['conc']>100])

In [ ]:
X,Y = ((gdf.x_wm.min(),gdf.x_wm.max()),(gdf.y_wm.min(),gdf.y_wm.max()))

In [ ]:
sum(Y)

In [ ]:
# we can now use bokeh to interactivly create new datashader images at different scales 
from bokeh.models import ColorBar, EqHistColorMapper
from bokeh.palettes import Spectral11

# p = simple_plot(background_fill_color=background)
p = simple_plot()
cvs = ds.Canvas(plot_width=plot_width, plot_height=plot_height, x_range=X, y_range=Y)
agg = cvs.points(gdf, 'x_wm', 'y_wm',ds.mean('conc'))
im = ds.transfer_functions.shade(agg, cmap=list(Spectral11))
# color_mapper = EqHistColorMapper(palette=Spectral11, nan_color="black")
color_mapper = EqHistColorMapper(palette=Spectral11)
p.image_rgba(image=[im.to_numpy()], x=X[0], y=Y[0], dw=plot_width*100, dh=plot_height*100)
# p.image(image=[agg], x=X[0], y=Y[0], dw=plot_width, dh=plot_height,color_mapper=color_mapper)
p.add_tile(Vendors.STAMEN_TERRAIN)

color_bar = ColorBar(color_mapper=color_mapper)
p.add_layout(color_bar, "right")

show(p)

In [ ]:
?p.image